In [1]:
with open("input.txt") as f:
    lines = [
        line.strip() for line in f.readlines()
    ]

Start by tokenizing each line so that we have a list
of single character tokens. I.e.

```
"1 + (2 * 3)"
```

Gets converted to

```
["1", "+", "(", "2", "*", "3", ")"]
```

In [2]:
import re
from typing import Callable, List, Tuple

Expression = List[str]

def tokenize(expr: str) -> Expression:
    """Turns exression string into list of tokens"""
    tokens_regex = "[\d*+\(\)]"
    return re.findall(tokens_regex, expr)

expressions: List[Expression] = [
    tokenize(line) for line in lines
]

Start with a recursive evaluator that we can use for
part 1 and part 2. The recursive evaluator takes a
tokenized expression and "flat evaluator" which evaluates
an expression that does not contain any parentheses
(this differs between part 1 and part 2).

The recursive evaluator looks for parentheses in the
expression. When it finds a set of parentheses, it
recursively evaluates the expression inside of the
parentheses and replaces the parentheses in the expression
with the result. When there are no more parentheses in
the expression, it calls the flat evaluator to evaluate
the remainder of the expression.

In [3]:
from copy import copy

def find_parens(
    expr: Expression
) -> Tuple[int, int]:
    """Returns a tuple with the indices of the first open
    and closing parentheses in an expression. The open
    parentheses index is the first open paren in the
    expression. The closing paren index is the matching
    closing paren for the opening paren (not necessarily
    the first closing paren in the expression).
    """
    open_index = expr.index("(")
    close_index = open_index
    # After finding the first open parentheses,
    # we need to find the matching closing parentheses.
    # This isn't necessarily the next closing parentheses
    # because there may be nested parentheses. So we
    # look through the rest of the expression, incrementing
    # a counter when we reach an open parentheses and
    # decrementing it when we reach a closing parentheses.
    # When the counter equals 0, we've found the matching
    # closing parentheses.
    paren_count = 1
    while paren_count != 0:
        close_index += 1
        paren_count += expr[close_index] == "("
        paren_count -= expr[close_index] == ")"

    return open_index, close_index

def evaluate(expr: Expression, flat_eval: Callable) -> int:
    """Takes an expression and a function that evaluates expressions
    without parentheses and returns the result of the evaluation.
    """
    expr = copy(expr)
    while "(" in expr:
        open_index, close_index = find_parens(expr)
        paren_expr = expr[open_index+1:close_index]
        paren_result = evaluate(paren_expr, flat_eval)
        expr[open_index:close_index+1] = [paren_result]

    # Once we no longer have parentheses in the expression, use the
    # flat evaluator to evaluate the remaining expression.
    return flat_eval(expr)

## part1

In [4]:
from collections import deque

def left_to_right_eval(expr: Expression) -> int:
    """Takes an expression which doesn't contain
    parentheses and evaluates it from left-to-right.
    """
    expr = deque(expr)
    # Use a stack-based approach where we pop 3
    # elements off of the stack. The first and last
    # elements are integers on either side of the
    # operator. The middle element is either a '+'
    # or '*'. Evaluate the expression formed by the
    # 3 elements and push the result back onto the
    # stack. When the stack length is 1, then the
    # remaning element is the result of the expression.
    while len(expr) > 1:
        left = int(expr.popleft())
        op = expr.popleft()
        right = int(expr.popleft())
        if op == "+":
            expr.appendleft(left + right)
        else:
            expr.appendleft(left * right)

    return expr[0]

sum(
    evaluate(expression, left_to_right_eval)
    for expression in expressions
)

3885386961962

## part2

In [5]:
def add_precedence_eval(expr: Expression) -> int:
    """Takes a flat expression and evaluates it such that
    addition has the highest precedence.
    """
    while "+" in expr:
        add_index = expr.index("+")
        left_index = add_index - 1
        right_index = add_index + 1
        left = int(expr[left_index])
        right = int(expr[right_index])
        expr[left_index:right_index+1] = [left + right]

    return left_to_right_eval(expr)

sum(
    evaluate(expression, add_precedence_eval)
    for expression in expressions
)

112899558798666